In [311]:
import os
import numpy as np
import pandas as pd
import time
import re
from selenium.webdriver import Chrome, ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

In [42]:
webdriver = 'chromedriver_win32/chromedriver.exe'

In [12]:
driver = Chrome(webdriver)

In [161]:
driver = Chrome(webdriver)#, options=chrome_options)
url = "https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/kernels"
driver.get(url)

In [418]:
actions = ActionChains(driver)
driver.find_element_by_class_name("dataset-header-v2__title").click()
while (driver.find_element_by_class_name("smart-list__message").text != 'No more notebooks to show'):
    actions.key_down(Keys.PAGE_DOWN).perform()

In [419]:
items = driver.find_elements_by_xpath('//*[@id="site-content"]/div[2]/div[2]/div/div/div/div[1]/div[2]/div[2]/div/div/div/div')
for each in items:
    print(each.find_element_by_class_name("kernel-list-item__name").text)

COVID-19 Case Study - Analysis, Viz & Comparisons
CoronaWhy.org - Global Collaboration (join slack)
Browsing research papers with a BM25 search engine
CORD-19: Explore Drugs Being Developed
Topic Modeling: Finding Related Articles
"Learning" Medicine with Word Embeddings ⚕️💊💉
COVID-19 Transmission and incubation
Visualization of Virus Origin & Genetic Evolution
Air Temperature and COVID-19
Anserini+BERT-SQuAD for context corpus search
COVID-19 Four different epidemic situations
covid-19-incubation-time
A CORD19 Research Paper Search Engine
Age Dependent Incubation Period
Asking questions with a BM25/BERT
Corona Virus Data Visualization
3D World Heat Map of Cumulative Confirmed Case
Exploratory Data Analysis covid-19 from eCDC
CORD-19 Metadata Enrich [2/x]: Altmetric API
Convert COVID-19 papers to Entity Vectors
CORD-19 Topic Modelling
NLP Text Mining - Disease behavior
COVID-2019-ID Paper version
CORD-19 Analysis with Sentence Embeddings
Utilizing BioBERT for K-means Topic Clustering
A

CORD-19: interactive word2vec paragraph search
COVID-19 (Task 01)
COVID-19 Word Embedding Approach
[CORD-19] Embeddings 📙 from abstracts with SpaCy
symptoms word cloud
Detecting and Visualizing covid-19
[CORD-19] Embeddings 📙 from abstracts with SpaCy
NG-EDA-COVID-19-v2
COVID-19 World Statistics
Extract Tika cTAKES Features
Covid-19 Analysis
Extractive text summarization
COVID-19: Extracting the Hidden Topics With Gensim
COVID-19 - Interactive Cluster Graph
Papers on Virus Genetics
Enigma Covid-19
BM25 Search + Query Similarity Ranking
COVID parameter study - India & others
naive LSTM model test for covid19
Search functions for cord19-challenge
Browsing research papers with a BM25 search engine
COVID-19 Dataset : Gaining actionable insights 📊
What Are The Treatments?
Tracking the spread of the Coronavirus
Clustering Papers using K-Means and t-SNE
Find similar COVID-19 Research Articles
COVID-19 Classify articles-Doc2vec PCA
Build CSV of Body Text
betweenness_centrality_of_the_bibliogra

In [413]:
print(items[23].get_attribute('innerHTML'))

<div class="kernel-list-item__meta"><div class="kernel-list-item__votes"><div class="vote-button__container "><div class="vote-button vote-button--enabled"><div class="vote-button__button vote-button__button--up vb-upvote"><span class="fa fa-caret-up"></span></div><div class="vote-button__button vote-button__button--up vote-button__vote-count-container"><span class="vote-button__vote-count">57</span></div><div class="vote-button__button-placeholder"></div></div></div></div><div class="kernel-list-item__image-wrapper"><span class="tooltip-container" data-tooltip="David Mezzetti"><a class="avatar" href="/nofoosports" style="width: 36px; position: relative; z-index: 2; pointer-events: auto;"><img class="avatar__thumbnail" src="https://storage.googleapis.com/kaggle-avatars/thumbnails/555186-kg.png" alt="David Mezzetti" width="36" height="36" style="border-radius: 3.6px;"><img class="avatar__tier" src="/static/images/avatier/avatier-contributor@2x.png" alt="contributor tier" width="36" styl

# Features
- votes
- user
- tier
- notebook_name
- time_published
- relative_time_published
- tags
- outputs
- language
- num_comments
- link

In [444]:
metadata = {'votes': [], 'user_link': [], 'tier': [], 'notebook_name': [], 
            'num_visualizations': [], 'num_datafiles': [], 'time_published': [], 
            'relative_time_published': [], 'tags': [], 'language': [], 
            'num_comments': [], 'notebook_link': []}

In [445]:
tidx = 6 #test index of items
print(items[tidx].find_element_by_class_name("vote-button__vote-count").text)
print(items[tidx].find_element_by_class_name('avatar').get_attribute("href"))
print(items[tidx].find_element_by_class_name('avatar__tier').get_attribute('alt').split(' ')[0])
print(items[tidx].find_element_by_class_name('kernel-list-item__name').text)
print(items[tidx].find_element_by_class_name('kernel-list-item__details').\
                  find_element_by_xpath('.//*[@title]').get_attribute('title'))
print(items[tidx].find_element_by_class_name('kernel-list-item__details').\
                  find_element_by_xpath('.//*[@title]').text)
print([each.text for each in items[tidx].find_elements_by_class_name('Tag_TextAnchor-sc-hezo17') if each.text != ''])
tooltip = items[tidx].find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[1]').get_attribute('data-tooltip')[:-1].split(' ')
                                                                     # remove the period
if ('visualizations' in tooltip) or ('visualization' in tooltip):
    for i, v in enumerate(tooltip):
        if ((v == 'visualizations' or v == 'visualization') and (tooltip[i-1].isdigit())):
            print('vis', tooltip[i-1])
        elif ((v == 'visualizations' or v == 'visualization') and (not tooltip[i-1].isdigit())):
            print('vis', 0)
else:
    print('vis', 0)
if ('data' in tooltip):
    for i, v in enumerate(tooltip):            
        if ((v == 'data') and (tooltip[i-1].isdigit())):
            print('data', tooltip[i-1])
        elif ((v == 'data') and (not tooltip[i-1].isdigit())):
            print('data', 0)
else:
    print('data', 0)
#always (visualizations then data files) when both present

print(items[tidx].find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[2]').get_attribute("data-tooltip").split(' ')[-1:][0])
print(re.sub('\D', '', items[tidx].find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[3]').get_attribute("data-tooltip")))
print(items[tidx].find_element_by_xpath('.//a').get_attribute('href'))

6
https://www.kaggle.com/ajrwhite
contributor
COVID-19 Transmission and incubation
Sat Mar 28 2020 13:50:24 GMT-0400 (Eastern Daylight Time)
4h ago
['covid19']
vis 0
data 1
Python
4
https://www.kaggle.com/ajrwhite/covid-19-transmission-and-incubation


In [ ]:
for i, each in enumerate(items):
    if (i%50==0):
        print(i)
    metadata['votes'].append(int(each.find_element_by_class_name("vote-button__vote-count").text))
    metadata['user_link'].append(each.find_element_by_class_name('avatar').get_attribute("href"))
    metadata['tier'].append(each.find_element_by_class_name('avatar__tier').get_attribute('alt').split(' ')[0])
    metadata['notebook_name'].append(each.find_element_by_class_name('kernel-list-item__name').text)
    tooltip = each.find_element_by_class_name('kernel-list-item__info-blocks')\
              .find_element_by_xpath('.//span[1]').get_attribute('data-tooltip')[:-1].split(' ')
    if ('visualizations' in tooltip) or ('visualization' in tooltip):
        for i, v in enumerate(tooltip):
            if ((v == 'visualizations' or v == 'visualization') and (tooltip[i-1].isdigit())):
                metadata['num_visualizations'].append(int(tooltip[i-1]))
            elif ((v == 'visualizations' or v == 'visualization') and (not tooltip[i-1].isdigit())):
                metadata['num_visualizations'].append(0)
    else:
        metadata['num_visualizations'].append(0)
    if ('data' in tooltip):
        for i, v in enumerate(tooltip):            
            if ((v == 'data') and (tooltip[i-1].isdigit())):
                metadata['num_datafiles'].append(int(tooltip[i-1]))
            elif ((v == 'data') and (not tooltip[i-1].isdigit())):
                metadata['num_datafiles'].append(0)
    else:
        metadata['num_datafiles'].append(0)
    metadata['time_published'].append(each.find_element_by_class_name('kernel-list-item__details').\
                                          find_element_by_xpath('.//*[@title]').get_attribute('title'))
    metadata['relative_time_published'].append(each.find_element_by_class_name('kernel-list-item__details').\
                                              find_element_by_xpath('.//*[@title]').text)
    metadata['tags'].append([each.text for each in\
                             each.find_elements_by_class_name('Tag_TextAnchor-sc-hezo17')\
                             if each.text != ''])
    metadata['language'].append(each.find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[2]').get_attribute("data-tooltip").split(' ')[-1:][0])
    metadata['num_comments'].append(re.sub('\D', '', each.find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[3]').get_attribute("data-tooltip")))
    metadata['notebook_link'].append(each.find_element_by_xpath('.//a').get_attribute('href'))

0
50
100
150
200
250
300
350
400


In [ ]:
metadata

In [ ]:
meta_df = pd.DataFrame.from_dict(metadata)

In [ ]:
meta_df

In [ ]:
meta_df.to_csv("covid-kernels.csv", index = False)